# 20 Newsgroups & Logistic Regression

This notebook is a slightly enhanced demo used during my presentation during the first, closed-beta Data Science Community meetup that took place on <br> 16 <sup>th</sup> of January 2017.

Here's shown how to use scikit-learn's Logistic Regression, and SGDClassifier (that is a more robust implementation of LR, mainly used for large datasets).

---

The used dataset is 20 Newsgroups, pre-vectorized using TF-IDF algorithm.

I've used a 5-fold cross validation scheme and Grid Search for hyperparameter tuning.

Also to get higher accuracy, feature vectors were scaled and normalized.

---

**Imports**

In [5]:
from __future__ import print_function

import os
import numpy as np
from sklearn import linear_model, preprocessing, cross_validation, datasets
import cPickle as cpk
from time import time

**Global variables**

In [6]:
LR_FILE_PATH = "models/lr.model"
SGD_FILE_PATH = "models/sgd.model"
LR_OP_FILE_PATH = "models/optimized_lr.model"
SGD_OP_FILE_PATH = "models/optimized_sgd.model"

**Loading the dataset**

In [7]:
start = time()
news = datasets.fetch_20newsgroups_vectorized()
print("Data loaded in {0} seconds".format(time() - start))
print("-------------------------------------------------------------------------------")

X = news.data
y = news.target

# _ = [print(topic) for topic in news.target_names] # you may uncoment this to see the topics

print("Number of documents in the dataset: {0}".format(*X.shape))
print("Size of the feature vector of a document: {1}".format(*X.shape))

Data loaded in 1.20432806015 seconds
-------------------------------------------------------------------------------
Number of documents in the dataset: 11314
Size of the feature vector of a document: 130107


**Some utility functions**

In [8]:
def to_str_repr(label):
    return news.target_names[label]

def formated_print(model_name, acc):
    pretty_stat = "Accuracy of the {0} model is {1}.".format(model_name, acc)
    print(pretty_stat)
    
def train_estimator(estimator, features):
    train, _ = next(iter(kfcv))
    estimator.fit(features[train], y[train])
    return estimator

def save_estimator(estimator, filename):
    """
        Note that it is recomended to use '.pkl' or '.model' file extension.
        
        Why? Because I want so.
        ;)
    """
    with open(filename, "wb") as fp:
        cpk.dump(estimator, fp)

def load_estimator(filename):
        with open(filename, "rb") as fp:
            return cpk.load(fp)

def load_or_train(estimator, feature_set, filename):
    if not os.path.exists(filename):
        print("Training...")
        start = time()
        estimator = train_estimator(estimator, feature_set)
        print("Trained in {0} seconds.".format(time() - start))
        print("-------------------------------------------------------------------------------")
        print("Saving...")
        start = time()
        save_estimator(estimator, filename)
        print("Saved in {0} seconds.".format(time() - start))
    else:
        print("Loading...")
        start = time()
        estimator = load_estimator(filename)
        print("Loaded in {0} seconds.".format(time() - start))
        
    return estimator
        
def evaluate_estimator(estimator, features):
    print("Evaluating model's accuracy...")
    start = time()
    score = np.mean(cross_validation.cross_val_score(estimator, features, y, cv = kfcv, n_jobs = 4))
    print("Evaluation done in {0} seconds".format(time() - start))
    print("-------------------------------------------------------------------------------")
    return score

**Feature vector normalization and scaling**

Due to the fact that the vectors are sparse, we must scale them just by the standard variance.

In [9]:
__X = preprocessing.scale(X, with_mean = False)
X_new = preprocessing.normalize(__X)

**5 Fold cross validation**

In order to get more objective results, I have used a 5 fold cross validation scheme, with feature-target pairs shuffling. Because of it, it is now possible to get an averaged result from 5 model evaluations, with training and testing on different portions of the dataset.

In [10]:
kfcv = cross_validation.KFold(len(y), n_folds = 5, shuffle = True)

**Estimators with default parameters**

Almost default, in order to speed up the training process I changed the number of workers from 1 to 3.

In [11]:
sgd = linear_model.SGDClassifier(n_jobs = 3)
lr = linear_model.LogisticRegression(n_jobs = 3)

**Accuracy benchmark for default estimators**

**Logistic Regression**

In [12]:
lr = load_or_train(lr, X, LR_FILE_PATH)

Loading...
Loaded in 3.42451095581 seconds.


In [13]:
mean_acc = evaluate_estimator(lr, X)

formated_print("Linear Regression", mean_acc)

Evaluating model's accuracy...
Evaluation done in 92.5229671001 seconds
-------------------------------------------------------------------------------
Accuracy of the Linear Regression model is 0.792645537933.


**SGD Classifier**

In [14]:
sgd = load_or_train(sgd, X, SGD_FILE_PATH)

Loading...
Loaded in 4.05311703682 seconds.


In [15]:
mean_acc = evaluate_estimator(sgd, X)

formated_print("Stochastic Gradient Descent", mean_acc)

Evaluating model's accuracy...
Evaluation done in 3.10582184792 seconds
-------------------------------------------------------------------------------
Accuracy of the Stochastic Gradient Descent model is 0.867332004143.


**Optimized estimators**

After playing around with Grid Search hyperparameter optimizer, I've found the below near optimial configuration.

In [16]:
sgd_op = linear_model.SGDClassifier(n_iter = 25, alpha = 0.00005, n_jobs = 3)
lr_op = linear_model.LogisticRegression(max_iter = 500, C = 3593.8136638046258, n_jobs = 3)

**Accuracy benchmark for optimized models**

**Logistic Regression**

In [ ]:
lr_op = load_or_train(lr_op, X_new, LR_OP_FILE_PATH)

Loading...
Loaded in 3.9715089798 seconds.


In [ ]:
mean_acc = evaluate_estimator(lr_op, X_new)

formated_print("Optimized Linear Regression", mean_acc)

Evaluating model's accuracy...


**SGD Classifier**

In [ ]:
sgd_op = load_or_train(sgd_op, X_new, SGD_OP_FILE_PATH)

In [ ]:
mean_acc = evaluate_estimator(sgd_op, X_new)

formated_print("Optimized Stochastic Gradient Descent", mean_acc)

# Conclusion

From all the evaluations done above I can conclude the following 2 things:
- Data preparation, in this case, scaling and normalizing feature vectors is extremely important and leads to major accuracy improvements.
- SGD Classifier is much faster than Logistic Regression while the accuracy is on par or slightly better, so it is preferable when working with big datasets.